## Week 2 Day 2

Our first Agentic Framework project!!

Prepare yourself for something ridiculously easy.

We're going to build a simple Agent system for generating cold sales outreach emails:
1. Agent workflow
2. Use of tools to call functions
3. Agent collaboration via Tools and Handoffs

## Before we start - some setup:


Please visit Sendgrid at: https://sendgrid.com/

(Sendgrid is a Twilio company for sending emails.)

If SendGrid gives you problems, see the alternative implementation using "Resend Email" in community_contributions/2_lab2_with_resend_email

Please set up an account - it's free! (at least, for me, right now).

Once you've created an account, click on:

Settings (left sidebar) >> API Keys >> Create API Key (button on top right)

Copy the key to the clipboard, then add a new line to your .env file:

`SENDGRID_API_KEY=xxxx`

And also, within SendGrid, go to:

Settings (left sidebar) >> Sender Authentication >> "Verify a Single Sender"  
and verify that your own email address is a real email address, so that SendGrid can send emails for you.


In [48]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace, function_tool
from openai.types.responses import ResponseTextDeltaEvent
from typing import Dict
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
import asyncio

# Email configuration - change these to your verified sender and recipient
SENDER_EMAIL = "johnny.jarecsni@icloud.com"
RECIPIENT_EMAIL = "johnny.jarecsni@gmail.com"


In [49]:
load_dotenv(override=True)

True

In [50]:
# Let's just check emails are working for you

def send_test_email():
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email(SENDER_EMAIL)
    to_email = To(RECIPIENT_EMAIL)
    content = Content("text/plain", "This is an important test email")
    mail = Mail(from_email, to_email, "Test email", content).get()
    response = sg.client.mail.send.post(request_body=mail)
    print(response.status_code)

send_test_email()

202


### Did you receive the test email

If you get a 202, then you're good to go!

#### Certificate error

If you get an error SSL: CERTIFICATE_VERIFY_FAILED then students Chris S and Oleksandr K have suggestions:  
First run this: `!uv pip install --upgrade certifi`  
Next, run this:
```python
import certifi
import os
os.environ['SSL_CERT_FILE'] = certifi.where()
```

#### Other errors or no email

If there are other problems, you'll need to check your API key and your verified sender email address in the SendGrid dashboard

Or use the alternative implementation using "Resend Email" in community_contributions/2_lab2_with_resend_email

(Or - you could always replace the email sending code below with a Pushover call, or something to simply write to a flat file)

## Step 1: Agent workflow

In [51]:
instructions1 = "You are a sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write professional, serious cold emails."

instructions2 = "You are a humorous, engaging sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write witty, engaging cold emails that are likely to get a response."

instructions3 = "You are a busy sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write concise, to the point cold emails."

In [52]:
sales_agent1 = Agent(
        name="Professional Sales Agent",
        instructions=instructions1,
        model="gpt-4o-mini"
)

sales_agent2 = Agent(
        name="Engaging Sales Agent",
        instructions=instructions2,
        model="gpt-4o-mini"
)

sales_agent3 = Agent(
        name="Busy Sales Agent",
        instructions=instructions3,
        model="gpt-4o-mini"
)

In [53]:

result = Runner.run_streamed(sales_agent1, input="Write a cold sales email")
async for event in result.stream_events():
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end="", flush=True)

Subject: Streamline Your SOC 2 Compliance Efforts

Dear [Recipient's Name],

I hope this message finds you well. I’m [Your Name], a representative from ComplAI, where we specialize in simplifying the complexities of SOC 2 compliance. 

In today’s fast-paced digital landscape, ensuring that your organization meets compliance standards is more crucial than ever. Our AI-powered SaaS tool not only automates the compliance process but also prepares you for audits with ease, saving you time and reducing stress.

Here are a few key benefits our platform offers:

- **Automated Documentation:** Streamline the creation and management of required documentation.
- **Audit Readiness:** Be prepared for audits with comprehensive reports and real-time data.
- **Continuous Monitoring:** Gain insights into compliance status at any time.

I would love to schedule a brief call to discuss how ComplAI can assist your organization in achieving and maintaining SOC 2 compliance effectively. Are you available f

In [54]:
message = "Write a cold sales email"

with trace("Parallel cold emails"):
    results = await asyncio.gather(
        Runner.run(sales_agent1, message),
        Runner.run(sales_agent2, message),
        Runner.run(sales_agent3, message),
    )

outputs = [result.final_output for result in results]

for output in outputs:
    print(output + "\n\n")


Subject: Streamline Your SOC 2 Compliance Process with ComplAI

Hi [Recipient's Name],

I hope this message finds you well. My name is [Your Name], and I’m reaching out to introduce you to ComplAI, a cutting-edge SaaS solution designed to simplify the SOC 2 compliance journey for businesses like yours.

Achieving and maintaining SOC 2 compliance can be a complex and time-consuming process. ComplAI leverages advanced AI technology to automate various aspects of compliance management, ensuring that your organization not only meets but exceeds industry standards with ease. Our platform offers:

- **Real-time monitoring** of compliance status
- **Automated documentation** preparation for audits
- **Continuous risk assessments** to stay ahead of potential issues

By implementing ComplAI, you can reduce the time and resources spent on manual compliance processes, allowing your team to focus on what truly matters: driving your business forward.

I would love to schedule a brief call to discus

In [55]:
sales_picker = Agent(
    name="sales_picker",
    instructions="You pick the best cold sales email from the given options. \
Imagine you are a customer and pick the one you are most likely to respond to. \
Do not give an explanation; reply with the selected email only.",
    model="gpt-4o-mini"
)

In [56]:
message = "Write a cold sales email"

with trace("Selection from sales people"):
    results = await asyncio.gather(
        Runner.run(sales_agent1, message),
        Runner.run(sales_agent2, message),
        Runner.run(sales_agent3, message),
    )
    outputs = [result.final_output for result in results]

    emails = "Cold sales emails:\n\n" + "\n\nEmail:\n\n".join(outputs)

    best = await Runner.run(sales_picker, emails)

    print(f"Best sales email:\n{best.final_output}")


Best sales email:
Subject: A SOC2 Ninja to Slice Through Your Compliance Worries! 🥷💼

Hi [Recipient's Name],

Ever feel like managing SOC2 compliance is like trying to assemble IKEA furniture blindfolded? Pieces everywhere, instructions in a foreign language, and at the end, you’re left wondering if it’ll collapse under pressure! 😅

Well, let me introduce you to ComplAI: your trusty sidekick in the quest for stress-free compliance. Our AI-powered tool doesn’t just help you prepare for audits—it practically rolls out the red carpet! 🎉

Here’s how we work our magic:

1. **Stress Less**: We automate the boring stuff, so you can focus on more critical tasks—like figuring out why your coffee machine is always broken. ☕
   
2. **Audit Ready**: We get you prepared to impress auditors, or as I like to call them, the compliance superheroes! 🦸‍♂️

3. **Peace of Mind**: With our AI, you can finally sleep soundly, knowing compliance won’t keep you up at night (unless you have that coffee machine a

Now go and check out the trace:

https://platform.openai.com/traces

## Part 2: use of tools

Now we will add a tool to the mix.

Remember all that json boilerplate and the `handle_tool_calls()` function with the if logic..

In [57]:
sales_agent1 = Agent(
        name="Professional Sales Agent",
        instructions=instructions1,
        model="gpt-4o-mini",
)

sales_agent2 = Agent(
        name="Engaging Sales Agent",
        instructions=instructions2,
        model="gpt-4o-mini",
)

sales_agent3 = Agent(
        name="Busy Sales Agent",
        instructions=instructions3,
        model="gpt-4o-mini",
)

In [58]:
sales_agent1

Agent(name='Professional Sales Agent', instructions='You are a sales agent working for ComplAI, a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. You write professional, serious cold emails.', prompt=None, handoff_description=None, handoffs=[], model='gpt-4o-mini', model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=None, truncation=None, max_tokens=None, reasoning=None, metadata=None, store=None, include_usage=None, extra_query=None, extra_body=None, extra_headers=None, extra_args=None), tools=[], mcp_servers=[], mcp_config={}, input_guardrails=[], output_guardrails=[], output_type=None, hooks=None, tool_use_behavior='run_llm_again', reset_tool_choice=True)

## Steps 2 and 3: Tools and Agent interactions

Remember all that boilerplate json?

Simply wrap your function with the decorator `@function_tool`

In [59]:
@function_tool
def send_email(body: str):
    """ Send out an email with the given body to all sales prospects """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email(SENDER_EMAIL)
    to_email = To(RECIPIENT_EMAIL)
    content = Content("text/plain", body)
    mail = Mail(from_email, to_email, "Sales email", content).get()
    sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}

### This has automatically been converted into a tool, with the boilerplate json created

In [60]:
# Let's look at it
send_email

FunctionTool(name='send_email', description='Send out an email with the given body to all sales prospects', params_json_schema={'properties': {'body': {'title': 'Body', 'type': 'string'}}, 'required': ['body'], 'title': 'send_email_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x1104ad260>, strict_json_schema=True, is_enabled=True)

### And you can also convert an Agent into a tool

In [61]:
tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description="Write a cold sales email")
tool1

FunctionTool(name='sales_agent1', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x1106d8900>, strict_json_schema=True, is_enabled=True)

### So now we can gather all the tools together:

A tool for each of our 3 email-writing agents

And a tool for our function to send emails

In [62]:
description = "Write a cold sales email"

tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description=description)
tool2 = sales_agent2.as_tool(tool_name="sales_agent2", tool_description=description)
tool3 = sales_agent3.as_tool(tool_name="sales_agent3", tool_description=description)

tools = [tool1, tool2, tool3, send_email]

tools

[FunctionTool(name='sales_agent1', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x1104adda0>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='sales_agent2', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent2_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x10ff3a2a0>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='sales_agent3', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required'

## And now it's time for our Sales Manager - our planning agent

In [63]:
# Improved instructions thanks to student Guillermo F.

instructions = """
You are a Sales Manager at ComplAI. Your goal is to find the single best cold sales email using the sales_agent tools.
 
Follow these steps carefully:
1. Generate Drafts: Use all three sales_agent tools to generate three different email drafts. Do not proceed until all three drafts are ready.
 
2. Evaluate and Select: Review the drafts and choose the single best email using your judgment of which one is most effective.
 
3. Use the send_email tool to send the best email (and only the best email) to the user.
 
Crucial Rules:
- You must use the sales agent tools to generate the drafts — do not write them yourself.
- You must send ONE email using the send_email tool — never more than one.
"""


sales_manager = Agent(name="Sales Manager", instructions=instructions, tools=tools, model="gpt-4o-mini")

message = "Send a cold sales email addressed to 'Dear CEO'"

with trace("Sales manager"):
    result = await Runner.run(sales_manager, message)

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/stop.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Wait - you didn't get an email??</h2>
            <span style="color:#ff7800;">With much thanks to student Chris S. for describing his issue and fixes. 
            If you don't receive an email after running the prior cell, here are some things to check: <br/>
            First, check your Spam folder! Several students have missed that the emails arrived in Spam!<br/>Second, print(result) and see if you are receiving errors about SSL. 
            If you're receiving SSL errors, then please check out theses <a href="https://chatgpt.com/share/680620ec-3b30-8012-8c26-ca86693d0e3d">networking tips</a> and see the note in the next cell. Also look at the trace in OpenAI, and investigate on the SendGrid website, to hunt for clues. Let me know if I can help!
            </span>
        </td>
    </tr>
</table>

### And one more suggestion to send emails from student Oleksandr on Windows 11:

If you are getting certificate SSL errors, then:  
Run this in a terminal: `uv pip install --upgrade certifi`

Then run this code:
```python
import certifi
import os
os.environ['SSL_CERT_FILE'] = certifi.where()
```

Thank you Oleksandr!

## Remember to check the trace

https://platform.openai.com/traces

And then check your email!!


### Handoffs represent a way an agent can delegate to an agent, passing control to it

Handoffs and Agents-as-tools are similar:

In both cases, an Agent can collaborate with another Agent

With tools, control passes back

With handoffs, control passes across



In [64]:

subject_instructions = "You can write a subject for a cold sales email. \
You are given a message and you need to write a subject for an email that is likely to get a response."

html_instructions = "You can convert a text email body to an HTML email body. \
You are given a text email body which might have some markdown \
and you need to convert it to an HTML email body with simple, clear, compelling layout and design."

subject_writer = Agent(name="Email subject writer", instructions=subject_instructions, model="gpt-4o-mini")
subject_tool = subject_writer.as_tool(tool_name="subject_writer", tool_description="Write a subject for a cold sales email")

html_converter = Agent(name="HTML email body converter", instructions=html_instructions, model="gpt-4o-mini")
html_tool = html_converter.as_tool(tool_name="html_converter",tool_description="Convert a text email body to an HTML email body")


In [65]:
@function_tool
def send_html_email(subject: str, html_body: str) -> Dict[str, str]:
    """ Send out an email with the given subject and HTML body to all sales prospects """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email(SENDER_EMAIL)
    to_email = To(RECIPIENT_EMAIL)
    content = Content("text/html", html_body)
    mail = Mail(from_email, to_email, subject, content).get()
    sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}

In [66]:
tools = [subject_tool, html_tool, send_html_email]

In [67]:
tools

[FunctionTool(name='subject_writer', description='Write a subject for a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'subject_writer_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x1106d8a40>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='html_converter', description='Convert a text email body to an HTML email body', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'html_converter_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x1106d99e0>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='send_html_email', description='Send out an email with the given subject and HTML body to all sales pros

In [68]:
instructions ="You are an email formatter and sender. You receive the body of an email to be sent. \
You first use the subject_writer tool to write a subject for the email, then use the html_converter tool to convert the body to HTML. \
Finally, you use the send_html_email tool to send the email with the subject and HTML body."


emailer_agent = Agent(
    name="Email Manager",
    instructions=instructions,
    tools=tools,
    model="gpt-4o-mini",
    handoff_description="Convert an email to HTML and send it")


### Now we have 3 tools and 1 handoff

In [69]:
tools = [tool1, tool2, tool3]
handoffs = [emailer_agent]
print(tools)
print(handoffs)

[FunctionTool(name='sales_agent1', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x1104adda0>, strict_json_schema=True, is_enabled=True), FunctionTool(name='sales_agent2', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent2_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x10ff3a2a0>, strict_json_schema=True, is_enabled=True), FunctionTool(name='sales_agent3', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': 

In [ ]:
# Improved instructions thanks to student Guillermo F.

sales_manager_instructions = """
You are a Sales Manager at ComplAI. Your goal is to find the single best cold sales email using the sales_agent tools.
 
Follow these steps carefully:
1. Generate Drafts: Use all three sales_agent tools to generate three different email drafts. Do not proceed until all three drafts are ready.
 
2. Evaluate and Select: Review the drafts and choose the single best email using your judgment of which one is most effective.
You can use the tools multiple times if you're not satisfied with the results from the first try.
 
3. Handoff for Sending: Pass ONLY the winning email draft to the 'Email Manager' agent. The Email Manager will take care of formatting and sending.
 
Crucial Rules:
- You must use the sales agent tools to generate the drafts — do not write them yourself.
- You must hand off exactly ONE email to the Email Manager — never more than one.
"""


sales_manager = Agent(
    name="Sales Manager",
    instructions=sales_manager_instructions,
    tools=tools,
    handoffs=handoffs,
    model="gpt-4o-mini")

message = "Send out a cold sales email addressed to Dear CEO from Alice"

# with trace("Automated SDR"):
#     result = await Runner.run(sales_manager, message)

### Remember to check the trace

https://platform.openai.com/traces

And then check your email!!

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/exercise.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Exercise</h2>
            <span style="color:#ff7800;">Can you identify the Agentic design patterns that were used here?<br/>
            What is the 1 line that changed this from being an Agentic "workflow" to "agent" under Anthropic's definition?<br/>
            Try adding in more tools and Agents! You could have tools that handle the mail merge to send to a list.<br/><br/>
            HARD CHALLENGE: research how you can have SendGrid call a Callback webhook when a user replies to an email,
            Then have the SDR respond to keep the conversation going! This may require some "vibe coding" 😂
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00bfff;">Commercial implications</h2>
            <span style="color:#00bfff;">This is immediately applicable to Sales Automation; but more generally this could be applied to  end-to-end automation of any business process through conversations and tools. Think of ways you could apply an Agent solution
            like this in your day job.
            </span>
        </td>
    </tr>
</table>

## Extra note:

Google has released their Agent Development Kit (ADK). It's not yet got the traction of the other frameworks on this course, but it's getting some attention. It's interesting to note that it looks quite similar to OpenAI Agents SDK. To give you a preview, here's a peak at sample code from ADK:

```
root_agent = Agent(
    name="weather_time_agent",
    model="gemini-2.0-flash",
    description="Agent to answer questions about the time and weather in a city.",
    instruction="You are a helpful agent who can answer user questions about the time and weather in a city.",
    tools=[get_weather, get_current_time]
)
```

Well, that looks familiar!

And a student has contributed a customer care agent in community_contributions that uses ADK.

---

# Part 4: Email Reply Capability 🎯

## HARD CHALLENGE SOLUTION!

This section implements automatic email reply handling using IMAP polling.

### How it works:
1. **Polls your inbox** every 60 seconds for new replies
2. **Analyzes sentiment** (interested/objection/question/not interested)
3. **Routes to specialized agents** based on reply type
4. **Sends intelligent responses** automatically
5. **Maintains conversation history** for context

### Setup Required:
See `EMAIL_REPLY_SETUP.md` for detailed instructions on getting your email app password.

In [ ]:
# Additional imports for email reply functionality
import imaplib
import email
from email.header import decode_header
import time
import json
from datetime import datetime
from pathlib import Path

# Email polling configuration - add these to your .env file
IMAP_SERVER = os.environ.get('IMAP_SERVER', 'imap.gmail.com')
IMAP_EMAIL = os.environ.get('IMAP_EMAIL', RECIPIENT_EMAIL)
IMAP_PASSWORD = os.environ.get('IMAP_PASSWORD')

# Create conversations directory
Path("conversations").mkdir(exist_ok=True)

print(f"📧 Email monitoring configured for: {IMAP_EMAIL}")
print(f"🔐 Password set: {'✅ Yes' if IMAP_PASSWORD else '❌ No - add IMAP_PASSWORD to .env'}")

📧 Email monitoring configured for: johnny.jarecsni@icloud.com
🔐 Password set: ✅ Yes


In [ ]:
# Conversation storage - regular functions first, then tool wrappers
def _store_conversation_impl(email_address: str, direction: str, subject: str, body: str) -> Dict[str, str]:
    """ Internal implementation for storing conversation """
    conversation = {
        "timestamp": datetime.now().isoformat(),
        "direction": direction,
        "subject": subject,
        "body": body
    }
    
    filename = f"conversations/{email_address.replace('@', '_at_').replace('<', '').replace('>', '')}.json"
    
    conversations = []
    if Path(filename).exists():
        with open(filename, 'r') as f:
            conversations = json.load(f)
    
    conversations.append(conversation)
    
    with open(filename, 'w') as f:
        json.dump(conversations, f, indent=2)
    
    return {"status": "stored", "count": len(conversations)}


def _get_conversation_history_impl(email_address: str) -> str:
    """ Internal implementation for getting conversation history """
    filename = f"conversations/{email_address.replace('@', '_at_').replace('<', '').replace('>', '')}.json"
    
    if not Path(filename).exists():
        return "No previous conversation history found."
    
    with open(filename, 'r') as f:
        conversations = json.load(f)
    
    history = []
    for conv in conversations:
        direction = "We sent" if conv["direction"] == "outbound" else "They replied"
        history.append(f"{direction} ({conv['timestamp']}):\nSubject: {conv['subject']}\n{conv['body'][:200]}...\n")
    
    return "\n---\n".join(history)


# Now create the tool wrappers
@function_tool
def store_conversation(email_address: str, direction: str, subject: str, body: str) -> Dict[str, str]:
    """ Store email in conversation history for context """
    return _store_conversation_impl(email_address, direction, subject, body)


@function_tool
def get_conversation_history(email_address: str) -> str:
    """ Retrieve previous emails exchanged with this prospect """
    return _get_conversation_history_impl(email_address)

print("✅ Conversation storage tools created")

✅ Conversation storage tools created


In [ ]:
def check_for_replies():
    """Poll email inbox for new replies using IMAP"""
    if not IMAP_PASSWORD:
        print("⚠️  IMAP_PASSWORD not set in .env file. Skipping email check.")
        return []
    
    try:
        mail = imaplib.IMAP4_SSL(IMAP_SERVER)
        mail.login(IMAP_EMAIL, IMAP_PASSWORD)
        mail.select("inbox")
        
        status, messages = mail.search(None, 'UNSEEN')
        if status != 'OK' or not messages[0]:
            return []
        
        replies = []
        message_ids = messages[0].split() if messages[0] else []
        
        if message_ids:
            print(f"📬 Found {len(message_ids)} unread email(s)")
        
        for num in message_ids:
            try:
                # Use BODY.PEEK[] to fetch without marking as read
                status, msg_data = mail.fetch(num, '(BODY.PEEK[])')
                
                if status != 'OK':
                    print(f"⚠️  Failed to fetch message {num}")
                    continue
                
                # msg_data format: [(b'1 (BODY[] {size}', email_bytes), b')']
                raw_email = None
                
                if isinstance(msg_data, list):
                    for item in msg_data:
                        if isinstance(item, tuple) and len(item) >= 2:
                            # Second element of tuple is the email
                            if isinstance(item[1], bytes) and len(item[1]) > 50:
                                raw_email = item[1]
                                break
                
                if not raw_email:
                    print(f"⚠️  Could not extract email from message {num}")
                    continue
                
                print(f"✅ Fetched email: {len(raw_email)} bytes")
                    
                msg = email.message_from_bytes(raw_email)
                
                from_email = msg.get("from", "") or msg.get("From", "")
                subject = msg.get("subject", "") or msg.get("Subject", "")
                
                print(f"📋 From: {from_email}")
                print(f"📋 Subject: {subject}")
                
                if not from_email:
                    print(f"⚠️  No from address, skipping")
                    continue
                
                # Decode subject
                if subject:
                    try:
                        decoded = decode_header(subject)[0]
                        if isinstance(decoded[0], bytes):
                            subject = decoded[0].decode(decoded[1] or 'utf-8')
                        else:
                            subject = str(decoded[0])
                    except:
                        pass
                
                # Extract body
                body = ""
                if msg.is_multipart():
                    for part in msg.walk():
                        if part.get_content_type() == "text/plain":
                            try:
                                payload = part.get_payload(decode=True)
                                if isinstance(payload, bytes):
                                    body = payload.decode('utf-8', errors='ignore')
                                elif isinstance(payload, str):
                                    body = payload
                                if body:
                                    break
                            except:
                                continue
                else:
                    try:
                        payload = msg.get_payload(decode=True)
                        if isinstance(payload, bytes):
                            body = payload.decode('utf-8', errors='ignore')
                        elif isinstance(payload, str):
                            body = payload
                        else:
                            body = str(msg.get_payload())
                    except:
                        body = str(msg.get_payload())
                
                print(f"📝 Body: {body[:100]}...")
                print(f"✅ Successfully parsed email\n")
                
                replies.append({
                    "from": from_email,
                    "subject": subject or "(no subject)",
                    "body": body[:1000] if body else ""
                })
            except Exception as e:
                print(f"⚠️  Error processing message {num}: {e}")
                import traceback
                traceback.print_exc()
                continue
        
        mail.close()
        mail.logout()
        return replies
        
    except Exception as e:
        print(f"❌ Error checking emails: {e}")
        import traceback
        traceback.print_exc()
        return []

print("✅ Email polling function created")


✅ Email polling function created


In [ ]:
reply_analyzer = Agent(
    name="Reply Analyzer",
    instructions="""You analyze customer replies to sales emails.
    
    Determine the sentiment and intent from these categories:
    - INTERESTED: Customer wants demo, call, more info, or shows positive interest
    - NOT_INTERESTED: Polite decline, not a fit, already have solution
    - OBJECTION: Has concerns about price, features, timing, or implementation
    - QUESTION: Needs clarification about product, pricing, or process
    - OUT_OF_OFFICE: Automated out-of-office reply
    
    Respond with ONLY the category name and a brief 1-sentence explanation.
    Example: "INTERESTED - Customer asked about scheduling a demo next week"
    """,
    model="gpt-4o-mini"
)

print("✅ Reply analyzer agent created")

✅ Reply analyzer agent created


In [ ]:
interested_responder = Agent(
    name="Interested Responder",
    instructions="""You write enthusiastic follow-up emails for interested prospects.
    - Express excitement about their interest
    - Offer specific next steps (calendar link, demo, call)
    - Keep the tone matching the original email style
    - Include 2-3 bullet points about what they'll learn/gain
    - End with clear call-to-action
    Sign off as 'The ComplAI Team'
    """,
    model="gpt-4o-mini"
)

objection_handler = Agent(
    name="Objection Handler",
    instructions="""You address customer concerns about ComplAI with empathy and solutions.
    - Acknowledge their concern specifically
    - Provide concrete solutions or alternatives
    - Include relevant proof points
    - Offer to discuss their specific situation
    - Keep tone helpful, not pushy
    Sign off as 'The ComplAI Team'
    """,
    model="gpt-4o-mini"
)

question_answerer = Agent(
    name="Question Answerer",
    instructions="""You answer questions about ComplAI clearly and helpfully.
    - Answer the specific question directly
    - Provide relevant details without overwhelming
    - Offer to schedule a call for complex questions
    - Suggest related features they might find useful
    Sign off as 'The ComplAI Team'
    """,
    model="gpt-4o-mini"
)

not_interested_handler = Agent(
    name="Not Interested Handler",
    instructions="""You write graceful responses to prospects who aren't interested.
    - Thank them for their time
    - Keep it brief (2-3 sentences)
    - Offer to stay in touch for future needs
    - Leave door open professionally
    Sign off as 'The ComplAI Team'
    """,
    model="gpt-4o-mini"
)

print("✅ All reply writer agents created")

✅ All reply writer agents created


In [ ]:
def _send_reply_email_impl(to_email: str, subject: str, body: str) -> Dict[str, str]:
    """ Internal implementation for sending reply email """
    print(f"📤 SENDING EMAIL:")
    print(f"  From: {SENDER_EMAIL}")
    print(f"  To: {to_email}")
    print(f"  Subject: {subject if subject.startswith('Re:') else 'Re: ' + subject}")
    print(f"  Body: {body[:200]}...")
    
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email(SENDER_EMAIL)
    to = To(to_email)
    
    if not subject.startswith("Re:"):
        subject = f"Re: {subject}"
    
    content = Content("text/plain", body)
    mail = Mail(from_email, to, subject, content).get()
    
    try:
        response = sg.client.mail.send.post(request_body=mail)
        print(f"✅ SendGrid response: {response.status_code}")
        _store_conversation_impl(to_email, "outbound", subject, body)
        return {"status": "sent", "code": response.status_code}
    except Exception as e:
        print(f"❌ SendGrid error: {e}")
        return {"status": "error", "message": str(e)}


@function_tool
def send_reply_email(to_email: str, subject: str, body: str) -> Dict[str, str]:
    """ Send a reply email maintaining the conversation thread """
    return _send_reply_email_impl(to_email, subject, body)

print("✅ Reply sending function created")


✅ Reply sending function created


In [ ]:
conversation_manager = Agent(
    name="Conversation Manager",
    instructions="""You manage ongoing email conversations with prospects.
    
    Follow these steps:
    1. Use get_conversation_history to understand the context
    2. Analyze the reply to determine intent
    3. Choose the appropriate responder agent based on intent
    4. Generate a response that references their specific points
    5. Send the reply using send_reply_email
    
    Be natural and helpful. Don't be pushy or robotic.
    """,
    tools=[
        get_conversation_history,
        reply_analyzer.as_tool("analyze_reply", "Analyze customer reply sentiment and intent"),
        interested_responder.as_tool("respond_interested", "Write response for interested prospect"),
        objection_handler.as_tool("handle_objection", "Address customer concerns"),
        question_answerer.as_tool("answer_question", "Answer customer questions"),
        not_interested_handler.as_tool("respond_not_interested", "Gracefully handle not interested"),
        send_reply_email
    ],
    model="gpt-4o-mini"
)

print("✅ Conversation manager agent created")

✅ Conversation manager agent created


In [ ]:
async def process_reply(from_email: str, subject: str, body: str):
    """Process an incoming email reply through the agent system"""
    print(f"\n{'='*60}")
    print(f"📧 Processing reply from: {from_email}")
    print(f"📋 Subject: {subject}")
    print(f"{'='*60}\n")
    
    _store_conversation_impl(from_email, "inbound", subject, body)
    
    context = f"""Customer Email: {from_email}
Subject: {subject}

Their Message:
{body}

Please analyze this reply and send an appropriate response."""
    
    try:
        with trace(f"Reply to {from_email}"):
            result = await Runner.run(conversation_manager, context)
        print(f"\n✅ Response sent successfully!")
        print(f"Agent output: {result.final_output[:200]}...")
    except Exception as e:
        print(f"\n❌ Error processing reply: {e}")

print("✅ Reply processing function created")

✅ Reply processing function created


In [ ]:
async def monitor_emails(check_interval=60, max_iterations=None):
    """Monitor inbox for new replies and process them automatically"""
    print("🚀 Starting email reply monitoring...")
    print(f"📬 Checking {IMAP_EMAIL} every {check_interval} seconds")
    print(f"⏹️  Press Ctrl+C to stop\n")
    
    iteration = 0
    
    try:
        while True:
            iteration += 1
            if max_iterations and iteration > max_iterations:
                print(f"\n🏁 Reached max iterations ({max_iterations})")
                break
            
            print(f"\n[{datetime.now().strftime('%H:%M:%S')}] Checking for new emails... (iteration {iteration})")
            replies = check_for_replies()
            
            if replies:
                print(f"\n🎯 Processing {len(replies)} new reply(ies)...")
                for reply in replies:
                    await process_reply(reply['from'], reply['subject'], reply['body'])
                    await asyncio.sleep(2)
            else:
                print("   No new replies")
            
            if max_iterations is None or iteration < max_iterations:
                print(f"   Sleeping for {check_interval} seconds...")
                await asyncio.sleep(check_interval)
    except KeyboardInterrupt:
        print("\n\n⏹️  Monitoring stopped by user")
    except Exception as e:
        print(f"\n\n❌ Error in monitoring loop: {e}")

print("✅ Email monitoring loop created")

✅ Email monitoring loop created


## Test with Manual Reply

Before starting the monitoring loop, test with a manual reply:

In [ ]:
# Test with a manual reply - sends to YOUR email so you can see it!
test_reply = {
    "from": "johnny.jarecsni@icloud.com",
    "subject": "Re: Transform Your Business with ComplAI",
    "body": "Hi, this sounds interesting! Can you tell me more about pricing and how long implementation typically takes?"
}

print("📧 Test Scenario:")
print(f"  Simulating reply from: {test_reply['from']}")
print(f"  Subject: {test_reply['subject']}")
print(f"  Message: {test_reply['body']}")
print(f"\n🤖 The agent will:")
print("  1. Analyze the sentiment (likely INTERESTED or QUESTION)")
print("  2. Generate an appropriate response")
print(f"  3. Send reply email to: {test_reply['from']}")
print(f"\n💡 You'll receive the reply at {test_reply['from']}!\n")

# Uncomment to test:
# await process_reply(test_reply['from'], test_reply['subject'], test_reply['body'])

print("✅ Test ready - uncomment the line above to run")

📧 Test Scenario:
  Simulating reply from: johnny.jarecsni@icloud.com
  Subject: Re: Transform Your Business with ComplAI
  Message: Hi, this sounds interesting! Can you tell me more about pricing and how long implementation typically takes?

🤖 The agent will:
  1. Analyze the sentiment (likely INTERESTED or QUESTION)
  2. Generate an appropriate response
  3. Send reply email to: johnny.jarecsni@icloud.com

💡 You'll receive the reply at johnny.jarecsni@icloud.com!

✅ Test ready - uncomment the line above to run


## Start Email Monitoring

**For testing:** Use `max_iterations=5` to limit checks  
**For production:** Remove `max_iterations` to run indefinitely

In [ ]:
# Start monitoring - this will check for emails every 30 seconds, 5 times
await monitor_emails(check_interval=30, max_iterations=2)

# For production, run indefinitely:
# await monitor_emails(check_interval=60)

🚀 Starting email reply monitoring...
📬 Checking johnny.jarecsni@icloud.com every 30 seconds
⏹️  Press Ctrl+C to stop


[13:02:05] Checking for new emails... (iteration 1)
📬 Found 1 unread email(s)
🔍 Fetch response for message b'3':
   Status: OK
   msg_data type: <class 'list'>
   msg_data length: 1
   Item 0: type=<class 'bytes'>, len=4
     Too small to be email: b'3 ()'
⚠️  Could not find email data in message b'3'
   No new replies
   Sleeping for 30 seconds...

[13:02:37] Checking for new emails... (iteration 2)
   No new replies

🏁 Reached max iterations (2)


## 🎉 Congratulations!

You've successfully implemented the HARD CHALLENGE!

### What you've built:
- ✅ IMAP email polling (no server required!)
- ✅ Multi-agent reply system
- ✅ Sentiment analysis and routing
- ✅ Conversation history tracking
- ✅ Intelligent response generation

### Next steps:
1. Check `conversations/` folder for stored email history
2. View agent decisions at https://platform.openai.com/traces
3. Send yourself a test email to see it in action
4. Customize agent instructions to match your style

Happy automating! 🚀